```
<Integrantes del Grupo>
<Sofia Oliverio
Ramiro Jimenez
Pablo Carrazana>
```

# Práctico 3 - Introducción al Aprendizaje Automático

## Modelos de Precios de Mercado
Queremos crear un modelo que prediga si el precio de una acción va a subir o bajar de acuerdo a información del pasado. Para ello implementaremos algunos modelos derivados de la teoria económica respecto de como valuar una activo con flujo de fondos inciertos, lo que estaría indicando que esas acciones tuvieron un rendimiento inferior al dado por  su índice de mercado(NASDAQ).


### Capital Asset Pricing Model (CAPM)

El modelo de CAPM propuesto por William Sharpe a partir del desarrollo de Markowitz sobre carteras eficientes, busca predecir el retorno de un activo en función del comportamiento global del mercado, y de la relación del activo puntual con el mercado. 

Según este modelo, que se puede expresar de la siguiente manera: 

$$
E(r_i) = r_f + \beta  [ E(r_m) - r_f ] 
$$

El retorno esperado del activo i, es igual al retorno libre de riesgo (o tasa libre de riesgo), más el exceso de retorno esperado de mercado (es decir lo que se espera que rinda el mercado por sobre la tasa libre de riesgo) escalado por un coeficiente beta. Este coeficiente beta es el conciente entre la covarianza del activo con el mercado sobre la varianza del mercado, y se interpreta como un multiplicador de riesgo de mercado.

Pueden ver una explicación completa en: https://economipedia.com/definiciones/modelo-valoracion-activos-financieros-capm.html

Adicionalmente, se puede agregar un término independiente como ordenada al origen alfa, que represanta una ganancia extraordinaria no explicada por la relacion con de la acción con el mercado sino por factores extraordinarios o arbitrages en caso de estrategias. De allí el famoso "buscar alfa" siendo una directriz de todos los portafolios managers. 

$$
E(r_i) = \alpha + r_f + \beta  [ E(r_m) - r_f ]
$$

Para realizar este ejercicio, deberán descargar el precio de una acción y del mercado asociado (por ejemplo GOOG y NASDAQ). En ambos casos tomar los retornos y tomar una tasa libre de riesgo. A continuación, deberán probar varias regresiones para comprobar el beta, y examinar si el agregado de una ordenada al origen aporta a los resultados. La idea es que seleccionen varias acciones para modelarlo con diferentes activos. 

Una vez que lo hayan hecho, adionalmente responder las siguientes preguntas:

1. ¿Es fácil encontrar $\alpha$ distinto de cero?

En nuestro caso los valores correspondientes al alpha de las acciones analizdas son distintos a cero por lo que no resulta dificil encontrar valores con estas características. Es importante aclarar que son valores cercanos a cero y para el caso de Alphabet Inc, Google, Microsoft y Facebook son valores negativas.

2. ¿Qué interpretación le podemos dar a $\beta$?

Beta es un parámetro que mide la sensibilidad del rendimiento de nuestras acciones respecto a las variaciones en la rentabilidad de mercado. Nos indica que tan volatil es nuestra acción en relación al mercado, es decir como se comporta entorno al riesgo sistémico.
3. ¿En qué momento nos gustaría buscar activos con $\beta$ > 1 y $\beta$ < 1? ¿Qué implica tener un $\beta$ < 0?

Un beta mayor a 1 indica que el precio de  nuestra acción reacciona más que proporcionalmente y en la misma dirección que los cambios ocurridos en los rendimientos de mercado. En un escenario de aumentos en los rendimientos de mercado uno desearía tener activos con esta característica.
Un beta menor a uno indica que el rendimiento de nuestra acción variará en menor porporción ante cambios en el rendimiento de mercado pero en igual dirección. Es decir en situaciones de caída de los mercados uno desearía que el rendimiento de su acción varíe menos que la variación en el mercado.

Un beta menor a cero indica que el rendimiento de nuestro activo tiene una relación inversa con el rendimiento del mercado. Esto puede darse por ejemplo con el índice VIX que mide la volatilidad del mercado y cuyo precio aumenta mientras mayor incertidumbre exista en el mercado.
$$
\alpha + \beta * SPY = KO
$$


In [1]:
! pip install yfinance
! pip install -U statsmodels==0.12.0

     |████████████████████████████████| 6.3 MB 43.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23918 sha256=d17035146a30141eee3eb75f83570d6080a171b9035a05d99f3918ff1825f922
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
     |████████████████████████████████| 9.5 MB 36.9 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [2]:
#Importamos librerías
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from pandas_datareader import data ##as pdr 
import yfinance as yf #importe esta librería porque me permitía trabajar con los datos de yahoo.
import datetime
import matplotlib.pyplot as plt
import pandas_datareader as pdr
import seaborn as sns
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.api import acf, pacf, graphics
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import explained_variance_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.model_selection import TimeSeriesSplit
yf.pdr_override()


In [3]:
# Definimos tickers en los cuales estamos interesados
# Algunas empresas del S&P500
tickers = ['MELI']

# Definimos el rango de fechas deseado
start_date = '2020-01-01'
end_date = '2020-12-31'

# Obtenemos un DataFrame
data_meli = data.get_data_yahoo(
    tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)["Adj Close"]

[*********************100%***********************]  1 of 1 completed


In [4]:
#La info la tengo organizada en una tupla 
data_meli
#La transformo a df
df=pd.DataFrame(data_meli)
df.head()

,Adj Close
Date,
2020-01-02,610.190002
2020-01-03,607.799988
2020-01-06,606.549988
2020-01-07,625.000000
2020-01-08,633.440002


In [5]:
#Renombro mi df
df_meli=df.copy()
df_meli.head()

,Adj Close
Date,
2020-01-02,610.190002
2020-01-03,607.799988
2020-01-06,606.549988
2020-01-07,625.000000
2020-01-08,633.440002


##### CAPM aplicado a la acción MELI

Como vimos anteriormente el CAPM es un modelo que describe la relación entre el rendimiento  esperado de nuestros activos y el riesgo inherente al mercado, denominado riesgo sistémico. Su objetivo es estimar el rendimiento esperado de estos activos dado este  riesgo sistémico. 
En este modelo el inversor diversifica su portfolio entre activos riesgosos y activos libre de riesgo.

Para aplicar CAPM a nuestra acción de Meli es necesario definir los retorno diarios de nuestra acción y un estimador de los retornos de mercado que en este caso es el SP500.

In [6]:
##Calculamos retornos diarios.
df_meli["ret_meli"]=df_meli['Adj Close'].pct_change(1)
df_meli.head()

,Adj Close,ret_meli
Date,,
2020-01-02,610.190002,NaN
2020-01-03,607.799988,-0.003917
2020-01-06,606.549988,-0.002057
2020-01-07,625.000000,0.030418
2020-01-08,633.440002,0.013504


In [7]:
#Obtenemos valores del sp500 para agregarloa nuestro df.
tickers = ['SPY']
data_spy = data.get_data_yahoo(
    tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)["Adj Close"]

df=pd.DataFrame(data_spy)
df["ret_spy"]=df['Adj Close'].pct_change(1)

df=pd.concat([df_meli,df],axis=1)

[*********************100%***********************]  1 of 1 completed


In [8]:
#Limpio los nulos pq despues me generan lío en la estimación.
df_fil=df.dropna(axis=0) 
df_fil.head()

,Adj Close,ret_meli,Adj Close,ret_spy
Date,,,,
2020-01-03,607.799988,-0.003917,314.437500,-0.007572
2020-01-06,606.549988,-0.002057,315.637115,0.003815
2020-01-07,625.000000,0.030418,314.749573,-0.002812
2020-01-08,633.440002,0.013504,316.427063,0.005330
2020-01-09,646.429993,0.020507,318.572632,0.006781


In [9]:
#Visualizamos la relación entre el rendimiento de nuestra acción y el rendimiento del mercado
import plotly.express as px
fig = px.scatter(df, x="ret_spy", y="ret_meli")
fig.show()

A primera vista podemos observar que hay una relación directa entre los rendimientos diarios analizados.

Calculamos alfa y beta para Meli:

In [10]:
beta, alpha = np.polyfit(df_fil["ret_spy"], df_fil["ret_meli"],1)
print(' El parametro beta para {} es = {} '.format('MELI',beta))
print(' El parametro alfa para {} es = {} '.format('MELI',alpha))

 El parametro beta para MELI es = 1.0442354184300864 
 El parametro alfa para MELI es = 0.003918557738519662 


Ahora trabajamos sobre los retornos del sp500.

In [11]:
#Promediamos sus rendimientos diarios:
spy_mean=df['ret_spy'].mean()
#Anualizamos este resultado:
rm=spy_mean*252
rm

0.21109683921507685

In [12]:
#Aplicamos formula
ER_meli=alpha + beta*rm
ER_meli

0.2243533539655441

Esto quiere decir que el modelo CAPM aplicado a nuestra acción de Meli, durante el período de tiempo analizado, indica que su rentabilidad esperada es del 22%.

Utilizamos este método para las acciones principales del Nasdaq.
Nuestro criterio será analizar las diez acciones con mayor capitalización de mercado.

In [13]:
#Descargo info de acciones que componen el nasdaq y los transformo en df.
df_nasdaq=pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQ4GS94-nvAEEpweAwIhzj_t7Q_j2MicJ9-oDROQehHiFQaxgN1Xqj0nrtkZWM7-U1RuwhrJlSChj3x/pub?output=csv')
df_nasdaq.head()


,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$2.88,-0.19,-6189%,9.129942e+07,China,NaN,123541,Miscellaneous,Service to the Health Industry
1,AAL,American Airlines Group Inc. Common Stock,$19.95,-0.11,-548%,1.291678e+10,United States,NaN,36473865,Transportation,Air Freight/Delivery Services
2,AAME,Atlantic American Corporation Common Stock,$4.25,0.04,0.95%,8.676707e+07,United States,NaN,30376,Finance,Life Insurance
3,AAOI,Applied Optoelectronics Inc. Common Stock,$7.62,-0.02,-262%,2.046891e+08,United States,2013.0,671346,Technology,Semiconductors
4,AAON,AAON Inc. Common Stock,$62.98,1.14,1843%,3.302367e+09,United States,NaN,193697,Capital Goods,Industrial Machinery/Components


In [14]:
#Ordeno y me quedo con diez acciones con valores de mark cap mas altos.
df_nasd_mc = df_nasdaq.sort_values('Market Cap',ascending=False)
df_nasdaq= df_nasd_mc[:10]


In [15]:
#Obtengo la lista de tickers a analizar
nasd_tickers=df_nasdaq['Symbol'].tolist()
nasd_tickers


['AAPL', 'MSFT', 'GOOG', 'GOOGL', 'AMZN', 'FB', 'TSLA', 'NVDA', 'PYPL', 'ASML']

In [16]:
#Agrego a mi  lista el ticker deñ indice nasdq
nasd_tickers.append('^IXIC')

In [17]:
#Creamos un data frame con el cierre ajustado de nuestras compañías.
df_tickers = data.DataReader(
    nasd_tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)['Adj Close']

df_tickers.head()

[*********************100%***********************]  11 of 11 completed


,AAPL,AMZN,ASML,FB,GOOG,GOOGL,MSFT,NVDA,PYPL,TSLA,^IXIC
Date,,,,,,,,,,,
2020-01-02,74.096443,1898.010010,299.266418,209.779999,1367.369995,1368.680054,157.903488,59.844322,110.750000,86.052002,9092.190430
2020-01-03,73.376083,1874.969971,294.443878,208.669998,1360.660034,1361.520020,155.937286,58.886452,108.760002,88.601997,9020.769531
2020-01-06,73.960770,1902.880005,292.101746,212.600006,1394.209961,1397.810059,156.340347,59.133400,110.169998,90.307999,9071.469727
2020-01-07,73.612930,1906.859985,295.274017,213.059998,1393.339966,1395.109985,154.914871,59.849308,109.669998,93.811996,9068.580078
2020-01-08,74.797081,1891.969971,298.337524,215.220001,1404.319946,1405.040039,157.382431,59.961559,111.820000,98.428001,9129.240234


In [18]:
#Obtengo rendimiento diario para cada ticker
#Defino una función que me permita recorrer las filas de cada columna  y calcular el rednimiento diario.

def retorno_diario(df): #Defino la fucion retorno diario que aplico sobre mi df
  df_retorno_diario = df.copy()
  for i in df.columns:
     for j in range(1, len(df)): #Para cada fila "j" en el rango definido opero:
         df_retorno_diario[i][j] = ((df[i][j]- df[i][j-1])/df[i][j-1]) * 100 
     df_retorno_diario[i][0] = 0 #La primer fila de cada columna le asigno el valor cero ya que tendrá el valor "NA".
  return df_retorno_diario

In [19]:
nasd_rd=retorno_diario(df_tickers)
nasd_rd

,AAPL,AMZN,ASML,FB,GOOG,GOOGL,MSFT,NVDA,PYPL,TSLA,^IXIC
Date,,,,,,,,,,,
2020-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-03,-0.972192,-1.213905,-1.611454,-0.529126,-0.490720,-0.523134,-1.245192,-1.600604,-1.796838,2.963319,-0.785519
2020-01-06,0.796835,1.488559,-0.795443,1.883360,2.465710,2.665406,0.258476,0.419363,1.296429,1.925466,0.562038
2020-01-07,-0.470303,0.209156,1.086016,0.216365,-0.062401,-0.193165,-0.911777,1.210666,-0.453844,3.880053,-0.031854
2020-01-08,1.608618,-0.780866,1.037513,1.013801,0.788033,0.711776,1.592849,0.187557,1.960428,4.920485,0.668905
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,-0.697602,-0.662712,-0.327612,0.381890,0.515231,0.465639,-1.303924,-2.025874,-1.663314,0.880775,-0.287397
2020-12-24,0.771218,-0.394945,1.784590,-0.264814,0.373473,0.343129,0.782731,-0.119131,-0.334114,2.444354,0.263251
2020-12-28,3.576578,3.507119,-0.589958,3.590130,2.141645,2.295055,0.992148,-0.721504,-1.219412,0.290128,0.739488


In [20]:
nasd_rd.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 252 entries, 2020-01-02 to 2020-12-30
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    252 non-null    float64
 1   AMZN    252 non-null    float64
 2   ASML    252 non-null    float64
 3   FB      252 non-null    float64
 4   GOOG    252 non-null    float64
 5   GOOGL   252 non-null    float64
 6   MSFT    252 non-null    float64
 7   NVDA    252 non-null    float64
 8   PYPL    252 non-null    float64
 9   TSLA    252 non-null    float64
 10  ^IXIC   252 non-null    float64
dtypes: float64(11)
memory usage: 23.6 KB


In [ ]:
#Visualizar  relación entre rendimientos de  nasdaq y estas acciones
#Sacamos fila 1.
nasd_rdf=nasd_rd.iloc[1:]
#Definimos una lista con las columnas del df anterior.
col_nasd=list(nasd_rdf.select_dtypes(include=['float64']).drop(['^IXIC'], axis = 1).columns)
#Aplicamos una función que nos permita graficar la relación entre rendimientos nasdaq y sus componentes.

for i in nasd_rd.columns:
  if i!= 'Date' and i != '^IXIC':
 # Use plotly express to plot the scatter plot for every stock vs. the S&P500
   fig = px.scatter(nasd_rd, x = '^IXIC', y = i, title = i)
    
   fig.show()

In [ ]:
#Creamos diccionarios vacíos
beta = {}
alpha = {}
#Calculamos beta y alpha para cada acción mediante una funcion
for i in nasd_rd.columns[:-1]:
  b,a = np.polyfit(nasd_rd['^IXIC'], nasd_rd[i],1)
  beta[i] = b    
  alpha[i] = a  
 

In [ ]:
#Cómo queda conformado el diccionario de betas de los tickers.
beta   

In [ ]:
#Cómo queda conformado el diccionario de alpha de los tickers seleccionados
alpha

In [ ]:
#Definimos acciones de nuestro diccionario.
tickers=list(beta.keys())
#Establecemos un diccionario donde se asignarán los valores de retornos de cada accion.
ER={}
ER_a={}
#Definimos tasa libre de riesgo.
rf=0
#Definimos retorno de mercado(promedio):
rm=nasd_rd['^IXIC'].mean() * 252 
#Definimos función de CAPM
for i in tickers:
 ER[i] = rf + (beta[i] * (rm-rf))


In [ ]:
#Analizamos los rendimientos de cada acción.
ER


In [ ]:
#Imprimimos los retornos esperados
for i in tickers:
  print('El modelo CAPM aplicado a {} indica que su rendimiento esperado es igual a {}%'.format(i, round(ER[i],3)))

### Modelos Autorregresivos para predecir precios.

Para esta parte del práctico, nos gustaría utilizar precios del pasado para poder estimar cuál va a ser el precio de una acción más adelante. Las regresiones que vemos en la introducción al aprendizaje automático no son las mismas que deben usarse cuando estamos trabajando con precios de mercado, ya que estos datos son series de tiempo. Para trabajar con este tipo de datos, es necesario generar **modelos autorregresivos**.

Si suponemos que $y_t$ es el precio de nuestra acción en el tiempo $t$, podemos generar el siguiente modelo autorregresivo:

$$
y_t = \delta + \phi_1 y_{t-1} + \dots + \phi_p y_{t-p} + \epsilon_t
$$

Existen otras variables que podemos agregar (como estacionariedad o tendencias), pero para este práctico podemos usar solamente el modelo `AutoReg` de la librería StatsModels: https://www.statsmodels.org/stable/examples/notebooks/generated/autoregressions.html.

El **ejercicio** para esta sección es crear una función que reciba 4 parámetros:
- El *ticker* de un activo.
- Una *fecha* a partir de la cual se van a querer predecir los precios del activo.
- Un *número de días* hacia atrás que se van a usar para ajustar el modelo autorregresivo.
- Un *número de días* hacia adelante para los cuales van a querer predecir el precio del activo utilizando el modelo entrenado.

Esta función se debe encargar de buscar los precios del ticker, realizar el ajuste y predecir el precio de la acción hacia delante. Además, debe devolver el [error cuadrático medio](https://es.wikipedia.org/wiki/Error_cuadr%C3%A1tico_medio) de la predicción, comparado con los verdaderos precios del activo.

¿Qué pasa si tratamos de predecir el precio del SPY el día antes del primer cisne negro de 2020 con un modelo que use 15 días hacia atrás?

In [ ]:
# Función que recibe 4 parámetros para predecir los retornos 
# de un activo a partir de las fechas

def predict_over_ticker_date(tickers, start_date, cant_days_before, cant_days_after):
   start_date_ticker= start_date - datetime.timedelta(days=cant_days_before)
   end_date_ticker= start_date + datetime.timedelta(days=cant_days_after)
  
   data_tic = data.get_data_yahoo(
     tickers,
     start=start_date_ticker, 
     end=end_date_ticker, 
     data_source='yahoo',
   )["Adj Close"]

   # Creo el data frame
   df=pd.DataFrame(data_tic)

   # Calculo los retornos
   df['Retornos']=df['Adj Close'].pct_change(1).dropna()

   # Elimino la columna Adj Close
   df.drop('Adj Close', inplace=True, axis=1)

   # Elimino los NaN
   df.dropna(inplace=True)

   # Separo test y train
   df_train=df[df.index <= start_date]
   df_test=df[df.index > start_date]

   # Calculo start y end para predicción con train con FECHAS
   start = df_train.index[0]
   end = df_train.index[-1]

   # Convierto el index de df_train
   df_train.index = pd.DatetimeIndex(df_train.index).to_period('D')
   df_test.index = pd.DatetimeIndex(df_test.index).to_period('D')

   # Instanciación
   mod = AutoReg(df_train, old_names=True, missing='drop', lags=1, hold_back=1)
   res = mod.fit()

   # Predicción del modelo con el start y end con valores de fechas
   predict = res.predict(df_test.index[0] , df_test.index[-1])

   # Imprimo el predict
   filtered_predict = predict[predict.index.isin(df_test.index)]
   
   # Calculo del error
   train_error = mean_squared_error(df_test["Retornos"], filtered_predict)
    
   # Devolvemos el predict y el error
   return predict, train_error


In [ ]:
# Función que recibe 4 parámetros para predecir los retornos 
# de un activo a partir de las fechas

def predict_over_ticker_date_price(tickers, start_date, cant_days_before, cant_days_after):
   start_date_ticker= start_date - datetime.timedelta(days=cant_days_before)
   end_date_ticker= start_date + datetime.timedelta(days=cant_days_after)
  
   data_tic = data.get_data_yahoo(
     tickers,
     start=start_date_ticker, 
     end=end_date_ticker, 
     data_source='yahoo',
   )["Adj Close"]

   # Creo el data frame
   df=pd.DataFrame(data_tic)

   # Elimino los NaN
   df.dropna(inplace=True)

   
   # Separo test y train
   df_train=df[df.index <= start_date]
   df_test=df[df.index > start_date]

   # Calculo start y end para predicción con train con FECHAS
   start = df_train.index[0]
   end = df_train.index[-1]

   # Convierto el index de df_train
   df_train.index = pd.DatetimeIndex(df_train.index).to_period('D')
   df_test.index = pd.DatetimeIndex(df_test.index).to_period('D')

   # Instanciación
   mod = AutoReg(df_train, old_names=True, missing='drop', lags=1, hold_back=1)
   res = mod.fit()

   # Predicción del modelo con el start y end con valores de fechas
   predict = res.predict(df_test.index[0] , df_test.index[-1])

   # Imprimo el predict
   filtered_predict = predict[predict.index.isin(df_test.index)]
   
   # Calculo del error
   train_error = mean_squared_error(df_test["Adj Close"], filtered_predict)
    
   # Devolvemos el predict y el error
   return predict, train_error

In [ ]:
# Definición de los valores para predecir los retornos y los precios de Meli en enero de 2021
start_date = datetime.datetime(year=2021, month=1, day=1)
cant_days_before = 15
cant_days_after = 15
tickers = ['MELI']

In [ ]:
# Llamamos a la función para predecir los retornos con fechas
predict_over_ticker_date(tickers, start_date, cant_days_before, cant_days_after)

In [ ]:
# Definición de los valores para predecir los retornos de SPY en marzo de 2021
start_date = datetime.datetime(year=2020, month=3, day=1)
cant_days_before = 15
cant_days_after = 15
tickers = ['SPY']

In [ ]:
# Llamamos a la función para predecir los retornos con fechas
predict_over_ticker_date(tickers, start_date, cant_days_before, cant_days_after)

In [ ]:
# Definición de los valores para predecir los precios de SPY en marzo de 2021
start_date = datetime.datetime(year=2020, month=3, day=1)
cant_days_before = 15
cant_days_after = 15
tickers = ['SPY']

In [ ]:
# Llamamos a la función para predecir los precios con fechas
predict_over_ticker_date_price(tickers, start_date, cant_days_before, cant_days_after)

¿Qué pasa si tratamos de predecir el precio del SPY el día antes del primer cisne negro de 2020 con un modelo que use 15 días hacia atrás?

In [ ]:
tickers = ['SPY']

#Se filtran datos de S&P500 desde 2000
start_date= '2020-01-01'
end_date= '2021-01-01'

dfP500 = data.DataReader(
    tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)["Adj Close"].to_frame()

In [ ]:
# Calcular los retornos para tickers del 2020
s = pd.Series(dfP500['Adj Close'].squeeze())
return_SP5000_year_2020 = (s.pct_change(periods=1, fill_method='pad', limit=None, freq=None))*100

# Calcular las desviación
std = return_SP5000_year_2020.std()

# Calcular la media de los retornos
mean = return_SP5000_year_2020.mean()

# Calcular el exceso de los retornos
excess_return_year_2020 = (mean - 0) / std

# Imprimir el describe de los retornos de los tickers del 2020
print("Describe de los retornos (Desde 2000):\n", return_SP5000_year_2020.describe())

# Imprimir el valor de la desviación
print("\nStd:", std)

# Imprimir el valor de la media
print("\nMean:", mean)

# Imprimir el exceso de los retornos
print("\nExceso de retornos:", excess_return_year_2020) 

In [ ]:
# Calcular 3 veces la desviación estándar
std3 =std * 3.0

# Imprimir el valor de 3 desviaciones estandar
print("Valor de 3 desviaciones estandar:", std3)

# Mínima y máxima para encontrar cisnes negros
mean_std_max = mean + std3
mean_std_min = mean - std3

# Imprimir valores mínimos y máximos de 3 desviaciones 
print("\nValores mínimos y máximos de 3 desviaciones")
print("Media desviación mayor", mean_std_max)
print("Media desviación menor", mean_std_min)

# Encontrar cisnes negros
list_cisnes = []
for retorno in return_SP5000_year_2020.iteritems(): 
    #Buscar los retornos mayores o menores a 3 desviaciones (+-) la media  
    if retorno[1] > mean_std_max or retorno[1] < mean_std_min: 
      list_cisnes.append(retorno)

In [ ]:
# Convertir la lista a un data frame
df_cisnes_negros = pd.DataFrame(list_cisnes)

# Asignar nombres a las columnas 
df_cisnes_negros.columns = ['Fecha','RetornoCisne']

# Formatear la fecha en datetime
df_cisnes_negros.Fecha = pd.to_datetime(df_cisnes_negros.Fecha)

# Se observan nuestros primeros cisnes negros
df_cisnes_negros.head(10)

Observamos nuestro primer cisne negro en el 2020 para el dia 2020-03-09

In [ ]:
dfP500[dfP500['Adj Close'].index > '2020-03-01'].head(15)

Al intentar predecir el precio del SPY un día antes del primer cisne negro de 2020 con un modelo que use 15 días hacia atrás ocurre que se predicen precios mayores a los verdaderos dado que en estos dias la accion tenia un valor mas alto y no era tan volatil

In [ ]:
start_date = datetime.datetime(year=2020, month=3, day=8)
cant_days_before = 15
cant_days_after = 15
tickers = ['SPY']

In [ ]:
predict_over_ticker_date_price(tickers, start_date, cant_days_before, cant_days_after)

### Árboles de Decisión para predecir suba o baja.

Venimos trabajando los precios con valores continuos, pero otra forma de verlos podría ser convirtiendolos en, por ejemplo, -1 y 1 para saber si el precio está en suba o en baja. De esta forma obtenemos un problema de clasificación que nos permite mirar los datos desde otra perspectiva.

- Etiqueten los datos con 1 si [pct_change](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html) es mayor a cero y 0, -1 si no.
- Entrenen un árbol de decisión que prediga si el valor de un activo va a subir o bajar.
- Reporten las siguientes métricas sobre el modelo resultante.

    - Accuracy
    - Precision
    - Recall
    - F1
    - matriz de confusión

In [ ]:
# Obtenemos un los tickers de Meli para la fecha entre 2020-01-01 y 2020-12_31
tickers= ['MELI']
start_date = '2020-01-01'
end_date = '2020-12-31'

data_meli = data.get_data_yahoo(
    tickers,
    start=start_date, 
    end=end_date, 
    data_source='yahoo',
)

# Creamos un dataframe con los datos de Meli
df_meli=pd.DataFrame(data_meli)

# Calculo el pct_change de meli
df_meli["ret_meli"]=df_meli['Adj Close'].pct_change(1)

# Elimino NAN
df_meli_model = df_meli.dropna()

# Muestro las primeras filas
df_meli_model.head()

In [ ]:
# Etiqueto en base a pct_change
df_meli_model['Target'] = df_meli_model.apply(lambda x: 1 if (x['ret_meli']) >= 0.0 else -1, axis=1)

# Elimino la columna pct_change
df_meli_model_filter = df_meli_model.drop(['ret_meli'],axis=1)
df_meli_model_filter.head()

In [ ]:
# División entre instancias y etiquetas
X, y = df_meli_model_filter.iloc[:, :6], df_meli_model_filter.Target
print(X)
tscv = TimeSeriesSplit(n_splits=2)
#print(tscv.split(X))

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

print(X_train)
print(X_test)

In [ ]:
# Entreno el modelo
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, y_train)

In [ ]:
# Predicción con los datos de train
y_predict_train = clf.predict(X_train)

# Preicción con los datos de test
y_predict_test = clf.predict(X_test)

# Impresión de los predict
print(classification_report(y_test, y_predict_test))
print(classification_report(y_train, y_predict_train))

# Mapa de calor para mostrar la eficencia del modelo
sns.heatmap(confusion_matrix(y_test, y_predict_test, normalize='true'),annot=True)

El etiquetado de datos no suele ser trivial dado a que depende de la naturaleza de los datos. Posiblemente, el precio de un activo suba un poco pero luego continue su declive ¿Cuánto tiene que subir/bajar un precio para ser considerado "suba"/"baja"?. ¿Tiene sentido tomar "periodos de suba/baja" más grandes para predecir si el comportamiento del activo en el futuro?. ¿Se podría utilizar un enfoque más estadístico para elegir el etiquetado?.

Aplicar GridSearch sobre los parámetros y luego para la mejor configuración encontrada, evaluar sobre el conjunto de entrenamiento y sobre el conjunto de evaluación, reportando:

- Accuracy
- Precision
- Recall
- F1
- matriz de confusión

In [ ]:
# Selección de parámetros
param_grid = {
    'criterion': ['entropy','gini'],
    'max_depth': [None,2,5,80],
    'min_samples_split' : [2,3,4]
}

prev_accuracy_score = 0
for params in ParameterGrid(param_grid):
    model = DecisionTreeClassifier(**params, random_state=0)
    model.fit(X_train, y_train)
    y_predict_test = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_predict_test, normalize=True)
    variance = explained_variance_score(y_test, y_predict_test, multioutput='uniform_average')
    print()
    print('Classification Report: ')
    print(classification_report(y_test, y_predict_test))
    print('Params: ',params)
    print('Accuracy: ', accuracy,' Variance: ', variance)
    print()
    if accuracy > prev_accuracy_score:
      best_model = model
      prev_accuracy_score = accuracy
      best_params = params
      best_variance = variance

print(best_model, prev_accuracy_score, best_params, best_variance)

In [ ]:
# Selección del mejor modelo
best_model = GridSearchCV(model, param_grid, scoring='accuracy', cv=5)
best_model.fit(X_train, y_train)

# Impresión de los valores de prección para train y para test con el mejor modelo
y_predict_test = best_model.predict(X_test)
y_predict_train = best_model.predict(X_train)

# Impresión del clasification report para los datos del mejor modelo
print(classification_report(y_test, y_predict_test))
print(classification_report(y_train, y_predict_train))

# Impresión de la matriz de confusión para comprender la eficacia del modelo
sns.heatmap(confusion_matrix(y_test, y_predict_test, normalize='true'),annot=True)